In [128]:
import json
import numpy as np
from dandi import dandiapi, dandiarchive
from remoteAssetIO import remoteAssetIO

In [219]:
def analyzeDataset(dandiID: str):
    
    LINE_UP = '\033[1A'
    LINE_CLEAR = '\x1b[2K'
    
    url = f"https://dandiarchive.org/dandiset/{dandiID}"
    print(url)
    
    devices = {}
    species = set()
    num_subjects = 0
    num_trials = []
    trial_times_mean = []

    parsed_url = dandiarchive.parse_dandi_url(url)
    with parsed_url.navigate() as (client, dandiset, assets):
    
        # Load the most recent version
        version = dandiset.most_recent_published_version
        if version is None: # => Draft dataset
            pass 
        dandiset = dandiset.for_version(version)
        
        print(dandiset.get_raw_metadata()['url'])
        #print(dandiset.get_raw_metadata()['assetsSummary']['numberOfSubjects'])
        print(dandiset.get_raw_metadata()['name'])
        
        # Number of assets seem to match number of files
        print(f"{len(list(dandiset.get_assets()))} assets")

        useful = True
        for i, asset in enumerate(assets):
            asset_md = asset.get_raw_metadata()

            fname = asset_md['path']
            size_mb = asset.size / (1024.**2)
            var_meas = [x['value'] for x in asset.get_raw_metadata()['variableMeasured']]

            # has Units or ElectricalSeries measurements
            useful = 'Units' in var_meas
            print(f"{i}: {fname[:40]} \t {size_mb:.3f}MB \t {useful}")
            
            if not useful:
                continue
                
            with remoteAssetIO(asset, "nwb-cache") as io:
                nwbfile = io.read()                
                nu = len(nwbfile.units)

                print(i, len(nwbfile.units.spike_times))
                
            
            #if useful:
            #    break

analyzeDataset("000409")

https://dandiarchive.org/dandiset/000409
https://dandiarchive.org/dandiset/000409/draft
IBL - Brain Wide Map
21 assets


KeyError: 'variableMeasured'

In [84]:
data

{'id': '000363',
 'version': Version(identifier='0.230124.1456', name='Mesoscale Activity Map Dataset', asset_count=174, size=65059242446, created=datetime.datetime(2023, 1, 24, 14, 56, 23, 709589, tzinfo=datetime.timezone.utc), modified=datetime.datetime(2023, 1, 24, 14, 56, 29, 612003, tzinfo=datetime.timezone.utc), status=<VersionStatus.VALID: 'Valid'>),
 'devices': ['OBIS470',
  '17131309211 (neuropixels 1.0 - 3B)',
  '18005107812 (neuropixels 1.0 - 3B)',
  '17216703352 (neuropixels 1.0 - 3B)',
  '18005107871 (neuropixels 1.0 - 3B)'],
 'species': ['Mus musculus']}